<a href="https://colab.research.google.com/github/123sh/Research-Paper_Summarizer/blob/main/Text_summarizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
import csv
df=pd.read_csv("/content/scisumm.csv")

In [12]:
df.head()

,text,summary
0,TnT - A Statistical Part-Of-Speech Tagger Trig...,TnT - A Statistical Part-Of-Speech Tagger\nTri...
1,Mildly Non-Projective Dependency Structures Sy...,Mildly Non-Projective Dependency Structures\nS...
2,Using Corpus Statistics And WordNet Relations ...,Using Corpus Statistics And WordNet Relations ...
3,Automatic Labeling Of Semantic Roles present a...,Automatic Labeling Of Semantic Roles\nWe prese...
4,Generative Models For Statistical Parsing With...,Generative Models For Statistical Parsing With...


In [14]:
df=df.rename(columns={"text":"source_text","summary":"target_text"})

In [15]:
df.head(3)

,source_text,target_text
0,TnT - A Statistical Part-Of-Speech Tagger Trig...,TnT - A Statistical Part-Of-Speech Tagger\nTri...
1,Mildly Non-Projective Dependency Structures Sy...,Mildly Non-Projective Dependency Structures\nS...
2,Using Corpus Statistics And WordNet Relations ...,Using Corpus Statistics And WordNet Relations ...


**Data Preparation for Summarization**

In [16]:
#add Prefix for summarization :PreFix ,"summarize"
df["source_text"]="summarize:"+df["source_text"]

In [17]:
df["source_text"].iloc[0]

"summarize:TnT - A Statistical Part-Of-Speech Tagger Trigrams'n'Tags (TnT) is an efficient statistical part-of-speech tagger. Contrary to claims found elsewhere in the literature, we argue that a tagger based on Markov models performs at least as well as other current approaches, including the Maximum Entropy framework. A recent comparison has even shown that TnT performs significantly better for the tested corpora. We describe the basic model of TnT, the techniques used for smoothing and for handling unknown words. Furthermore, we present evaluations on two corpora. A large number of current language processing systems use a part-of-speech tagger for pre-processing. The tagger assigns a (unique or ambiguous) part-ofspeech tag to each token in the input and passes its output to the next processing level, usually a parser. Furthermore, there is a large interest in part-ofspeech tagging for corpus annotation projects, who create valuable linguistic resources by a combination of automatic

Split the dataSet for train and **test**

In [18]:
train_df,test_df=train_test_split(df,test_size=0.3)

In [19]:
train_df.shape,test_df.shape

((706, 2), (303, 2))

**Use SimpleT5 Model**

In [22]:
!pip install simplet5 -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 60.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incomp

In [23]:
from simplet5 import SimpleT5

**Model Creation**

In [25]:
model=SimpleT5()

In [26]:
model.from_pretrained(model_type="t5",model_name="t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

***Model Training***

In [28]:
model.train(train_df=train_df,
            eval_df=test_df,
            source_max_token_len=128,
            target_max_token_len=512,
            batch_size=8,max_epochs=5,
            use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Model Infere

In [30]:
#let's Load the model
model.load_model("t5","/content/outputs/simplet5-epoch-4-train-loss-1.868-val-loss-1.8484")

**Result**

In [34]:
test_summary="summarize:"+"""The high mortality rate in Tuberculosis (TB) burden regions has increased significantly
in the last decades. Despite the possibility of treatment for TB, high burden regions still
suffer inadequate screening tools, which result in diagnostic delay and misdiagnosis.
These challenges have led to the development of Computer-Aided Diagnostic (CAD)
system to detect TB automatically. There are several ways of screening for TB, but Chest
X-Ray (CXR) is more prominent and recommended due to its high sensitivity in detecting
lung abnormalities. This paper presents the results of a systematic review based on
PRISMA procedures that investigate state-of-the-art Deep Learning techniques for
screening pulmonary abnormalities related to TB. The systematic review was conducted
using an extensive selection of scientific databases as reference sources that grant
access to distinctive articles in the field. Four scientific databases were searched to
retrieve related articles. Inclusion and exclusion criteria were defined and applied to each
article to determine those included in the study. Out of the 489 articles retrieved, 62
were included. Based on the findings in this review, we conclude that CAD systems are
promising in tackling the challenges of the TB epidemic and made recommendations for
improvement in future studies.
Keywords: tuberculosis, chest radiograph, computer-aided diagnosis, deep learning, systematic review
INTRODUCTION
Tuberculosis (TB) is ranked among the leading causes of death. About 10 million persons fell
ill globally from TB infections in 2019 (1). TB is triggered by the Mycobacterium bacteria that
usually affect the lungs (pulmonary) but sometimes affect other parts of the body (extrapulmonary)
(2). Many TB patients lose their lives yearly due to diagnostic delay, misdiagnosis, and lack of
appropriate treatments (3, 4). Although TB is a global challenge, the mortality rate is more prevalent
in low and middle-income nations (5).
TB is certainly treatable if diagnosed early for appropriate treatment. Early diagnosis is essential
for successful treatment, preventing further spread, and significantly reducing the mortality rate in
line with the World Health Organization (WHO) End TB Strategy (1). The gold standard for TB
screening is Sputum culture. However, posterior-anterior chest radiographs (CXR) are an effective
technique with low-cost and moderately low radiation doses for screening lung abnormalities
to achieve prompt results (6). CXR has been adequately employed in developed countries to
analyze individuals exhibiting active TB symptoms. At the same time, its application is limited
in developing countries where TB is most prevalent (7, 8). High TB burden regions lack the skilled
and radiological expertise required to interpret CXR images adequately (9, 10).
Oloko-Oba and Viriri Deep Learning Techniques for Tuberculosis Detection
In the last decades, several efforts have been made using
Artificial Intelligence (AI) to develop a Computer-Aided
Detection (CAD) system to advance automatic object/image
recognition tasks and overcome the challenges of a skilled
workforce. Machine Learning (ML) and Deep Learning (DL)
are the predominant AI techniques employed to develop CAD
systems for analyzing CXR images. Both techniques have had a
significant impact, but the DL approach, such as Convolutional
Neural Network (CNN), has become more prominent for
analyzing different pulmonary abnormalities in the medical
domain, most importantly in diagnosing TB. The application of
an efficient classification tool is vital for improving the quality of
diagnosis while reducing the time taken to analyze a large volume
of CXRs (11). This endeavor is to achieve the global decline in
TB incidence to about 5% annually compared to the current 2%
yearly as part of the World Health Organization strategy to end
TB (1).
The contribution of this systematic review is to present
an extensive summary of the various state-of-the-art CAD
system proposed in the literature for the classification of
TB. Ultimately, only the CAD system developed using Deep
Learning models is considered in this study detailing the
diagnostic accuracy between 2017 and 2021. The rest of
the paper is structured as follows: section Methodology
presents the study methodology. The results are presented in
section Results. Section Discussion presents the discussion,
while the conclusion is expressed in section Conclusion
and Recommendations.
FIGURE 1 | The PRISMA structure for the study selection process.
METHODOLOGY
This systematic review aims to establish various CAD systems
related to Tuberculosis diagnosis from CXR using DL techniques.
The study followed the Preferred Reporting Items for Systematic
Reviews and Meta-Analyses (PRISMA) procedures (12) to
identify the standards for inclusion and exclusion, as shown in
Figure 1. These standards were formulated based on the present
study objectives and the research questions. All articles that
satisfied the following conditions were included and excluded
if otherwise:
• Articles that considered only pulmonary tuberculosis disease.
• Employed at least one deep learning technique as a classifier
• CXR is the only medical imagine for screening tuberculosis.
• Articles published between January 2017 and September 2021.
• Articles are entirely written in English.
• Articles must be full text. All others, such as abstract, preprints
are excluded."""


In [36]:
len(test_summary)

5441

In [35]:
model.predict(test_summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (1156 > 512). Running this sequence through the model will result in indexing errors


['The high mortality rate in Tuberculosis (TB) burden regions has increased significantly in the last decades. Despite the possibility of treatment for TB, high burden regions still suffer inadequate screening tools, which result in diagnostic delay and misdiagnosis. These challenges have led to the development of Computer-Aided Diagnostic (CAD) system to detect TB automatically. Chest X-Ray (CXR) is more prominent and recommended due to its high sensitivity in detecting lung abnormalities related to TB. This systematic review was conducted using an extensive selection of scientific databases as reference sources that grant access to distinctive articles in the field. Out of the 489 articles retrieved, 62 were included. Based on the findings in this review, we conclude that CAD systems are promising in tackling the challenges of the TB epidemic.']